# Crawling #1: 가장 기본적인 형태의 크롤링/파싱

In [1]:
# 가장 기본적인 형태의 크롤링, 그리고 파싱에 대해서 배워볼 예정입니다.
# 이 방식을 이용해서 정적인 사이트 ( <=> 동적인 사이트; AJAX나 FrameSet 형태를 제외한; )
# 를 쉽게 크롤링하실 수 있습니다.

# 예제에서는, 
# 네이버 블로그를 크롤링하는 방법에 대해서 살펴보겠습니다.

In [2]:
import pandas as pd

import requests  # 크롤링을 위한 라이브러리
from bs4 import BeautifulSoup  # 파싱을 위한 라이브러리

In [3]:
post_df = pd.DataFrame(columns=[
    "검색 키워드",
    "블로그 제목",
    "블로그 이름",
    "블로그 주소",
])

In [4]:
# 일단은 검색할 키워드를 미리 선정합니다.
# 여러분들이 아마 데이터 분석을 하실 예정이면, 이런 키워드 또한 크롤링, 혹은 어떠한 방식으로 생성하여
# 크롤링을 수행해야 합니다

# 이렇게 할 수도 있겠지만, 아래처럼 한번 키워드 리스트를 만들어보겠습니다.
keywords = [
    "신사동 원룸",
    "신사역 원룸",
    "가로수길 원룸",
    "가로수길 오피스텔",
    "신사역 부동산",
]

# 지역 키워드
location_keywords = [
    "신사동",
    "신사역",
    "압구정동",
    "압구정역",
    "가로수길",
]

# 접미사 키워드
postfix_keywords = [
    "부동산",
    "원룸",
    "오피스텔",
    "투룸",
    "방",
]

keywords = [
    "{location_keyword} {postfix_keyword}".format(
        location_keyword=location_keyword,
        postfix_keyword=postfix_keyword
    )
    for location_keyword in location_keywords
    for postfix_keyword in postfix_keywords
]

keywords

['신사동 부동산',
 '신사동 원룸',
 '신사동 오피스텔',
 '신사동 투룸',
 '신사동 방',
 '신사역 부동산',
 '신사역 원룸',
 '신사역 오피스텔',
 '신사역 투룸',
 '신사역 방',
 '압구정동 부동산',
 '압구정동 원룸',
 '압구정동 오피스텔',
 '압구정동 투룸',
 '압구정동 방',
 '압구정역 부동산',
 '압구정역 원룸',
 '압구정역 오피스텔',
 '압구정역 투룸',
 '압구정역 방',
 '가로수길 부동산',
 '가로수길 원룸',
 '가로수길 오피스텔',
 '가로수길 투룸',
 '가로수길 방']

In [5]:
for keyword in keywords:
    # 우리가 작성한 키워드는 " " ( 띄어쓰기 ) 로 되어 있지만,
    # 네이버에서 검색을 할 떄는 " " => "+" 로 변경되서 검색되어야 합니다.
    
    query = keyword.replace(' ', '+')
    
    response = requests.get("https://search.naver.com/search.naver?query={query}".format(query=query))
    assert response.status_code is 200

    dom = BeautifulSoup(response.content, "html.parser")

    blog_post_elements = dom.select('li.sh_blog_top')

    for blog_post_element in blog_post_elements:
        title = blog_post_element.select('a.sh_blog_title')[0].attrs.get('title')
        blog_name = blog_post_element.select('a.txt84')[0].text
        url = blog_post_element.select('a.sh_blog_title')[0].attrs.get('href')
        
        post_df.loc[len(post_df)] = [keyword, title, blog_name, url]

In [6]:
post_df

,검색 키워드,블로그 제목,블로그 이름,블로그 주소
0,신사동 부동산,[신사동부동산] 세인파트너스 사무실인테리어 공사,강남부동산거래소,http://blog.naver.com/superb4u?Redirect=Log&lo...
1,신사동 부동산,[신사동상가] 강남 대형스튜디오 사무실 추천,강남부동산거래소,http://blog.naver.com/superb4u?Redirect=Log&lo...
2,신사동 부동산,강남구 신사동 상가 월세 5천/290만 - 가로수길 초입 [신사동부동산 - 신세계부동산],신세계 부동산,http://qstarq.blog.me/220624582269
3,신사동 원룸,신사동빌라 원룸 가로수길 축복하우스,(주)피플부동산&삼성2호점,http://blog.naver.com/people_vs?Redirect=Log&l...
4,신사동 원룸,신사동 원룸 풀옵션 압구정역 현대썬앤빌,(주)피플부동산&정효선,http://blog.naver.com/people_re?Redirect=Log&l...
5,신사동 원룸,강남구용달이사 강남원룸이사 논현동 신사동,☎ 1 6 7 0 - 9 9 8 6 친절이사,http://blog.daum.net/yk_6060/592589
6,신사동 오피스텔,신사역 오피스텔 샤르망s 신사동 전세,House Channel 02.566.2027,http://blog.daum.net/gangnamjeil/1391
7,신사동 오피스텔,신사동 압구정역 오피스텔 크레신타워 월세,아름다운 세상 TEL.02-555-4472 (24시간),http://blog.daum.net/arangsin/672
8,신사동 오피스텔,★은평구 신사동 오피스텔★ 응암역 정원탑오피스텔(복층형) 월세,"은평구 응암동 원룸, 오피스텔 월세 전세",http://blog.daum.net/heechul588/897
9,신사동 오피스텔,신사동 오피스텔 압구정역 현대썬앤빌,하우스마켓,http://blog.daum.net/gnhmk/1545


In [7]:
post_df.to_csv("naver_room.csv", index=False)